
# LangChain Agent Mini-Project

This notebook walks through building a simple research assistant agent with [LangChain](https://python.langchain.com). It is designed for hands-on classroom demos and runs in Google Colab.



## Project Overview

We will:

1. Install the dependencies that Colab needs for LangChain.
2. Explore the idea of tool-using agents and configure a Wikipedia search tool.
3. Build a ReAct-style agent that decides when to query Wikipedia and when to answer directly.
4. Run the agent with either a real OpenAI chat model (for live demos) or a built-in fake model (for offline/test runs).
5. Package helper utilities so your class can experiment with their own questions.



> **Tip for instructors:** The notebook defaults to a self-contained fake model so that it runs without external API keys. To use a real LLM on Colab, set the `OPENAI_API_KEY` environment variable (or edit the configuration cell) and switch the `USE_FAKE_MODEL` flag to `False`.


In [ ]:
# If you are running this in Colab, execute this cell once to install dependencies.
# On Colab this takes ~1 minute. Locally you may comment this out if packages already exist.
!pip install -qU "langchain==0.1.20" "langchain-community==0.0.38" "langchain-openai==0.1.7" wikipedia


## 1. Imports and configuration

We toggle between a fake model (great for automated tests or offline runs) and an OpenAI chat model. The fake model replies with pre-scripted ReAct traces so the rest of the workflow behaves exactly like a live demo.


In [ ]:

import os

USE_FAKE_MODEL = os.environ.get("USE_FAKE_LANGCHAIN_MODEL", "True").lower() in {"1", "true", "yes", "on"}
print(f"Using fake model: {USE_FAKE_MODEL}")


In [ ]:
if USE_FAKE_MODEL:
    from langchain_community.chat_models import FakeListChatModel

    fake_plan = """Thought: I should check the Wikipedia tool for background information.
Action: Wikipedia
Action Input: LangChain"""
    fake_summary = """Thought: I can answer the question now that I have context.
Final Answer: LangChain was created by Harrison Chase to help developers build agents and other LLM-powered applications by combining prompts, tools, and memory."""

    llm = FakeListChatModel(responses=[fake_plan, fake_summary])
else:
    from langchain_openai import ChatOpenAI

    if not os.environ.get("OPENAI_API_KEY"):
        raise ValueError("OPENAI_API_KEY is not set. Either provide a key or keep USE_FAKE_MODEL enabled.")

    llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

llm


## 2. Define tools the agent can use

We'll use LangChain's Wikipedia integration so the agent can look up facts. Tools can wrap APIs, local functions, or other chains. The agent will decide when to call the tool.


In [ ]:
from typing import Dict

from langchain_core.tools import Tool

if USE_FAKE_MODEL:
    SAMPLE_WIKI_SNIPPETS: Dict[str, str] = {
        "langchain": (
            "LangChain is an open-source framework created by Harrison Chase that helps developers build applications "
            "powered by large language models (LLMs). It focuses on orchestrating prompts, tools, memory, and agents."
        ),
        "harrison chase": (
            "Harrison Chase is the CEO and co-founder of LangChain. Prior to LangChain he worked on robotics and applied AI "
            "projects, and created the framework to simplify building LLM agents."
        ),
    }

    def offline_wikipedia(query: str) -> str:
        key = query.strip().lower()
        for term, blurb in SAMPLE_WIKI_SNIPPETS.items():
            if term in key:
                return blurb
        return (
            "Offline demo mode: pretend we looked this up on Wikipedia. "
            "LangChain streamlines connecting LLMs with tools so they can reason and act."
        )

    wikipedia_tool = Tool(
        name="Wikipedia",
        func=offline_wikipedia,
        description="Offline-friendly Wikipedia lookup for well-known AI topics."
    )
else:
    from langchain_community.utilities import WikipediaAPIWrapper
    from langchain_community.tools import WikipediaQueryRun

    wikipedia = WikipediaAPIWrapper(lang="en", top_k_results=1, doc_content_chars_max=2000)

    wikipedia_tool = Tool(
        name="Wikipedia",
        func=WikipediaQueryRun(api_wrapper=wikipedia).run,
        description="Useful for researching well-known topics or people by querying Wikipedia."
    )

TOOLS = [wikipedia_tool]
print("Configured tools:")
for tool in TOOLS:
    print(f"- {tool.name}: {tool.description}")


## 3. Create a ReAct-style agent

LangChain's `initialize_agent` helper wires the model and tools together. The ReAct (Reasoning + Acting) agent type alternates between reasoning steps ("Thought") and tool usage ("Action"/"Observation").


In [ ]:

from langchain.agents import AgentType, initialize_agent

agent_executor = initialize_agent(
    tools=TOOLS,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

agent_executor



## 4. Run the agent on a demo question

By default, the fake model produces a deterministic trace so you can narrate each step. When using a real OpenAI key, the agent will reach out to Wikipedia live and craft a new answer.


In [ ]:
demo_question = "Who created LangChain and why is it useful for building AI agents?"
result = agent_executor.invoke({"input": demo_question})

print("Final answer:", result["output"])


## 5. Helper function for interactive exploration

Use `run_agent` to let students supply their own questions. When `USE_FAKE_MODEL` is `True`, the function replays the deterministic trace. Flip the flag (and set your API key) for live responses.


In [ ]:

def run_agent(question: str) -> str:
    '''Run the LangChain agent and return the final answer.'''
    response = agent_executor.invoke({"input": question})
    return response["output"]

# Example usage (feel free to edit the question):
run_agent("What problem does LangChain solve for developers?")



## 6. Wrap-up ideas for class discussion

* Ask students to add another tool (e.g., a calculator or a news API) and observe how the agent behaves.
* Compare the deterministic fake trace with real OpenAI outputs to highlight the role of the LLM.
* Discuss guardrails: what instructions could we add to the system prompt to keep the agent focused on class-safe content?

Happy teaching!
